# Apply DOV
Based on training, a reasonable # missing is ~15

In [6]:
import pandas as pd
import pickle as pk
from functools import partial


from rdkit.Chem import AllChem

DOV_DICT = pk.load(
    open("DOV_4degree_256nBits_05dpm.pk", "rb")
)

DPM = 0.05
RADIUS = 4
NBITS=256

In [7]:
def get_num_missing_features_full(smi, pm, dpm, DOV_dict, radius=4, nBits=256):
    num_missing_features = 0
    
    m = AllChem.MolFromSmiles(smi)
    fp = AllChem.GetHashedMorganFingerprint(m, radius=radius, nBits=nBits)
    
    for i, key in enumerate(fp.GetNonzeroElements()):
        if key not in DOV_dict:
            num_missing_features += 1
        else:
            matching_pm = False
            for pm_loc in DOV_dict[key]:
                min_pm = pm_loc-dpm
                max_pm = pm_loc+dpm
                if min_pm <= pm and pm <= max_pm:
                    matching_pm = True
                    break
            
            if not matching_pm:
                num_missing_features += 1
    
    return num_missing_features

get_num_missing_features = partial(get_num_missing_features_full, dpm=DPM, DOV_dict=DOV_DICT, radius=RADIUS, nBits=NBITS)